In [1]:
import pandas as pd

In [36]:
adult_df = pd.read_csv("adult.csv")
income_df = pd.read_csv("income.csv")
demographic_df = pd.read_csv("US_PopAgeStruct_20230713030811.csv")

/var/folders/7w/xv3k68j12xg711gh35rpkxw40000gn/T/ipykernel_80562/3923488575.py:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  demographic_df = pd.read_csv("US_PopAgeStruct_20230713030811.csv")


In [39]:
poland_inflation = income_df.loc[income_df['country_name'] == 'Poland']
poland_inflation = poland_inflation.drop(['country_name', 'indicator_name', '2024'], axis=1)

In [16]:
X_income = income_df.drop("2024", axis=1)
y_income = income_df["2024"]

In [15]:
y_income

2        6.8
4       22.3
5        2.9
9        4.0
10       3.7
       ...  
187      5.9
189      5.7
190    200.0
191      3.4
194      9.6
Name: 2024, Length: 136, dtype: float64

In [20]:
X_income

,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
2,9.7,14.6,6.6,7.8,6.3,10.4,14.0,5.9,5.9,9.2,...,2.9,4.8,6.4,5.6,4.3,2.0,2.4,7.2,9.3,9.0
4,46.7,1.4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,7.3,9.2,30.7,29.8,19.6,17.1,22.3,25.8,21.4,13.1
5,19.0,11.5,4.2,2.3,3.8,1.0,0.5,3.6,6.8,4.4,...,1.1,1.0,-0.5,2.4,1.2,1.4,1.1,1.6,7.5,5.0
9,10.1,9.5,11.4,10.0,4.0,6.7,9.1,8.5,7.3,7.6,...,2.5,1.5,1.3,2.0,1.9,1.6,0.9,2.8,6.6,5.8
10,6.3,6.8,5.4,3.3,5.7,3.2,1.7,1.4,1.9,2.2,...,1.5,0.8,1.0,2.2,2.1,1.5,1.4,2.8,8.6,7.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,63.5,34.0,19.0,49.2,55.3,72.2,76.4,63.6,62.2,80.4,...,8.9,8.7,9.6,6.2,7.6,7.9,9.8,7.7,9.1,6.1
189,11.2,26.8,6.7,1.7,5.5,1.1,4.8,16.0,8.8,7.7,...,0.8,2.5,0.8,3.1,2.4,2.7,5.3,2.3,7.0,9.3
190,21.4,16.2,9.6,6.2,12.2,11.4,11.5,28.1,29.5,84.5,...,62.2,121.7,254.9,438.1,65374.1,19906.0,2355.1,1588.5,186.5,360.0
191,25.2,69.6,95.4,49.5,64.9,91.6,453.5,360.4,374.4,95.8,...,4.1,0.6,2.7,3.5,3.5,2.8,3.2,1.8,3.2,3.4


In [19]:
X_income = X_income.drop(['country_name', 'indicator_name'], axis=1)

In [26]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')

X_income = imputer.fit_transform(X_income)

y_income.fillna(y_income.median(), inplace=True)

/var/folders/7w/xv3k68j12xg711gh35rpkxw40000gn/T/ipykernel_80562/1220409280.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_income.fillna(y_income.median(), inplace=True)


In [27]:
from sklearn.model_selection import train_test_split

X_income_train, X_income_test, y_income_train, y_income_test = train_test_split(X_income, y_income)

In [28]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

def print_best_model_scoring(grid_search, X_test, y_test, model_name="Ridge Regression"):
    """
    Print comprehensive scoring metrics for the best model from GridSearchCV
    
    Parameters:
    - grid_search: Fitted GridSearchCV object
    - X_test: Test features
    - y_test: True test target values
    - model_name: Name of the model for display purposes
    """
    
    best_model = grid_search.best_estimator_
    
    y_pred = best_model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    

    n = len(y_test)
    p = X_test.shape[1]
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    
    print("=" * 60)
    print(f"SCORING RESULTS: {model_name}")
    print("=" * 60)
    print(f"Best Parameters: {grid_search.best_params_}")
    print(f"Best Cross-Validation Score (Negative MSE): {grid_search.best_score_:.4f}")
    print("\n--- TEST SET PERFORMANCE ---")
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"R-squared (R²): {r2:.4f}")
    print(f"Adjusted R-squared: {adj_r2:.4f}")
    print("\n--- INTERPRETATION ---")
    print(f"MSE: Lower is better (0 is perfect)")
    print(f"RMSE: In same units as target variable")
    print(f"MAE: Average absolute error")
    print(f"R²: {r2*100:.1f}% of variance explained")
    print("=" * 60)
    
    return {
        'best_model': best_model,
        'best_params': grid_search.best_params_,
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'adj_r2': adj_r2
    }

def print_comprehensive_scoring(grid_search, X_train, X_test, y_train, y_test, model_name="Ridge Regression"):
    """
    Print comprehensive scoring including training and test performance
    """
    
    best_model = grid_search.best_estimator_
    

    y_pred_train = best_model.predict(X_train)
    y_pred_test = best_model.predict(X_test)
    
    train_mse = mean_squared_error(y_train, y_pred_train)
    train_rmse = np.sqrt(train_mse)
    train_r2 = r2_score(y_train, y_pred_train)
    
    test_mse = mean_squared_error(y_test, y_pred_test)
    test_rmse = np.sqrt(test_mse)
    test_r2 = r2_score(y_test, y_pred_test)
    
    overfitting_gap = train_mse - test_mse
    
    print("=" * 70)
    print(f"COMPREHENSIVE SCORING: {model_name}")
    print("=" * 70)
    print(f"Best Parameters: {grid_search.best_params_}")
    print(f"Best CV Score (Negative MSE): {grid_search.best_score_:.4f}")
    print("\n--- TRAINING SET ---")
    print(f"MSE: {train_mse:.4f} | RMSE: {train_rmse:.4f} | R²: {train_r2:.4f}")
    print("\n--- TEST SET ---")
    print(f"MSE: {test_mse:.4f} | RMSE: {test_rmse:.4f} | R²: {test_r2:.4f}")
    print(f"\nOverfitting Gap (Train MSE - Test MSE): {overfitting_gap:.4f}")
    
    if overfitting_gap > 0:
        print("Model might be underfitting (train MSE > test MSE)")
    elif overfitting_gap < -train_mse * 0.1:  # If test MSE is significantly worse
        print("Model might be overfitting")
    else:
        print("Model shows good generalization")
    print("=" * 70)
    
    return {
        'best_model': best_model,
        'best_params': grid_search.best_params_,
        'train_mse': train_mse,
        'test_mse': test_mse,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'overfitting_gap': overfitting_gap
    }

In [29]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

ridge_income = Ridge(random_state=42)

param_grid_income = {
    'alpha': np.logspace(-3, 3, 13),
    'solver': ['auto', 'cholesky', 'sag']
}

cv_income = GridSearchCV(
    estimator=ridge_income,
    param_grid=param_grid_income,
    n_jobs=-1,
    verbose=1,
    scoring='neg_mean_squared_error'
)


cv_income.fit(X_income_train, y_income_train)

results_basic = print_best_model_scoring(cv_income, X_income_test, y_income_test, "Ridge Regression - Income Prediction")

results_comp = print_comprehensive_scoring(cv_income, X_income_train, X_income_test, y_income_train, y_income_test, "Ridge Regression - Income Prediction")

Fitting 5 folds for each of 39 candidates, totalling 195 fits


/Users/michalkawa/heackyeh-bieda-team/server/.venv/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/michalkawa/heackyeh-bieda-team/server/.venv/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/michalkawa/heackyeh-bieda-team/server/.venv/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/michalkawa/heackyeh-bieda-team/server/.venv/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/michalkawa/heackyeh-bieda-team/server/.venv/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The ma

SCORING RESULTS: Ridge Regression - Income Prediction
Best Parameters: {'alpha': np.float64(1000.0), 'solver': 'sag'}
Best Cross-Validation Score (Negative MSE): -16469.5732

--- TEST SET PERFORMANCE ---
Mean Squared Error (MSE): 39.8500
Root Mean Squared Error (RMSE): 6.3127
Mean Absolute Error (MAE): 4.5331
R-squared (R²): -0.2350
Adjusted R-squared: 4.7051

--- INTERPRETATION ---
MSE: Lower is better (0 is perfect)
RMSE: In same units as target variable
MAE: Average absolute error
R²: -23.5% of variance explained
COMPREHENSIVE SCORING: Ridge Regression - Income Prediction
Best Parameters: {'alpha': np.float64(1000.0), 'solver': 'sag'}
Best CV Score (Negative MSE): -16469.5732

--- TRAINING SET ---
MSE: 243.1069 | RMSE: 15.5919 | R²: 0.6176

--- TEST SET ---
MSE: 39.8500 | RMSE: 6.3127 | R²: -0.2350

Overfitting Gap (Train MSE - Test MSE): 203.2569
Model might be underfitting (train MSE > test MSE)


/Users/michalkawa/heackyeh-bieda-team/server/.venv/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [40]:
best_ridge_model = cv_income.best_estimator_

new_predictions = best_ridge_model.predict(poland_inflation)
print(f"\nSample predictions: {new_predictions}")
print(f"Actual values: {y_income_test.values[:5]}")


Sample predictions: [7.27237753]
Actual values: [3.4 4.7 6.6 5.6 1.5]


/Users/michalkawa/heackyeh-bieda-team/server/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but Ridge was fitted without feature names
  warnings.warn(


In [42]:
import joblib

joblib.dump(best_ridge_model, 'infaltion_model.joblib')
with open('inflation_prediciton_dataset', 'w') as f:
    f.write(','.join(poland_inflation))